# NERSC Cluster Deploy Tutorial: Tuning Hyperparameters of a Distributed PyTorch Model with PBT using Ray Train & Tune

📖 [Back to Table of Contents](../README.md)<br>
<!-- ⬅ [Previous notebook](./ex_01_pytorch_ray_hvd.ipynb) <br> -->
➡ [Next notebook](./ex_02_tensorflow_ray_train_tune.ipynb) <br>

----


## Introduction

We are going to run an example Ray Train & Tune code. This example looks at tunning hyperparameters of a distrbuted PyTorch Model with PBT. This tutorial is following the code in this example: https://docs.ray.io/en/latest/train/examples/pytorch/tune_cifar_torch_pbt_example.html

> **Note**:
> To setup the environment for the notebook, execute on command line: `./setup.sh 1` then select the kernel `pytorch-1.13.1` in the notebook

This Ray cluster will be setup using the NERSC PyTorch module and deployed on Perlmutter.



In [1]:
import sys

In [2]:
# !{sys.executable} -m pip install --upgrade git+https://github.com/asnaylor/nersc_cluster_deploy.git

## Creating Ray Cluster via login Node

To create a ray cluster on NERSC compute nodes, execute the `deploy_ray_cluster` function with your desired slurm sbatch options.

In [3]:
from nersc_cluster_deploy import deploy_ray_cluster
from utility import user_account

slurm_options = {
    'qos': 'debug',
    'account': user_account(),
    'nodes': '2',
    't': '00:30:00'
}
module_load = 'pytorch/1.13.1'

rayCluster = deploy_ray_cluster(
    slurm_options = slurm_options,
    job_setup = [f'module load {module_load}'] 
)

2023-05-04 14:20:27,417 INFO <Service-Manager> Starting up cluster
2023-05-04 14:20:27,418 INFO <RayHead> Starting service
2023-05-04 14:20:27,418 DEBUG <RayHead> Running cmd: /bin/bash -c "export RAY_GRAFANA_IFRAME_HOST=https://jupyter.nersc.gov/user/asnaylor/muller-shared-node-cpu/proxy/3000; module load pytorch/1.13.1;ray start --head --block --port 6379 --num-cpus=0 --num-gpus=0"
In case of issues, please refer to our known issues: https://docs.nersc.gov/current/
and open a help ticket if your issue is not listed: https://help.nersc.gov/


2023-05-04 14:20:28,616	INFO usage_lib.py:435 -- Usage stats collection is disabled.
2023-05-04 14:20:28,616	INFO scripts.py:710 -- Local node IP: 128.55.173.46
2023-05-04 14:20:30,602	SUCC scripts.py:747 -- --------------------
2023-05-04 14:20:30,602	SUCC scripts.py:748 -- Ray runtime started.
2023-05-04 14:20:30,602	SUCC scripts.py:749 -- --------------------
2023-05-04 14:20:30,602	INFO scripts.py:751 -- Next steps
2023-05-04 14:20:30,603	INFO scripts.py:752 -- To connect to this Ray runtime from another node, run
2023-05-04 14:20:30,603	INFO scripts.py:755 --   ray start --address='128.55.173.46:6379'
2023-05-04 14:20:30,603	INFO scripts.py:771 -- Alternatively, use the following Python code:
2023-05-04 14:20:30,603	INFO scripts.py:773 -- import ray
2023-05-04 14:20:30,603	INFO scripts.py:777 -- ray.init(address='auto')
2023-05-04 14:20:30,603	INFO scripts.py:790 -- To see the status of the cluster, use
2023-05-04 14:20:30,603	INFO scripts.py:791 --   ray status
2023-05-04 14:20:3

2023-05-04 14:20:37,465 INFO <Prometheus> Starting service
2023-05-04 14:20:37,466 DEBUG <Prometheus> Running cmd: shifter --image=prom/prometheus:v2.42.0 --volume=/mscratch/sd/a/asnaylor/ray_cluster/prometheus:/prometheus /bin/prometheus --config.file=/tmp/ray/session_latest/metrics/prometheus/prometheus.yml --storage.tsdb.path=/prometheus
2023-05-04 14:20:37,475 INFO <Grafana> Starting service
2023-05-04 14:20:37,475 DEBUG <Grafana> Running cmd: shifter --image=grafana/grafana-oss:9.4.3 --volume=/mscratch/sd/a/asnaylor/ray_cluster/grafana:/grafana --env GF_PATHS_DATA=/grafana --env GF_PATHS_PLUGINS=/grafana/plugins --env GF_SERVER_ROOT_URL=https://jupyter.nersc.gov/user/asnaylor/muller-shared-node-cpu/proxy/3000/ --env GF_PATHS_CONFIG=/tmp/ray/session_latest/metrics/grafana/grafana.ini --env GF_PATHS_PROVISIONING=/tmp/ray/session_latest/metrics/grafana/provisioning --entrypoint &
2023-05-04 14:20:37,483 INFO <Service-Manager> Cluster startup complete


Creating Ray workers via sbatch
Submitted batch job 259611


ts=2023-05-04T21:20:38.287Z caller=main.go:512 level=info msg="No time or size retention was set so using the default time retention" duration=15d
ts=2023-05-04T21:20:38.287Z caller=main.go:556 level=info msg="Starting Prometheus Server" mode=server version="(version=2.42.0, branch=HEAD, revision=225c61122d88b01d1f0eaaee0e05b6f3e0567ac0)"
ts=2023-05-04T21:20:38.287Z caller=main.go:561 level=info build_context="(go=go1.19.5, platform=linux/amd64, user=root@c67d48967507, date=20230201-07:53:32)"
ts=2023-05-04T21:20:38.287Z caller=main.go:562 level=info host_details="(Linux 5.14.21-150400.24.46_12.0.71-cray_shasta_c #1 SMP Sun Apr 30 16:36:43 UTC 2023 (92112fd) x86_64 login01 )"
ts=2023-05-04T21:20:38.287Z caller=main.go:563 level=info fd_limits="(soft=500000, hard=500000)"
ts=2023-05-04T21:20:38.287Z caller=main.go:564 level=info vm_limits="(soft=unlimited, hard=unlimited)"
ts=2023-05-04T21:20:38.293Z caller=web.go:561 level=info component=web msg="Start listening for connections" addres

logger=settings t=2023-05-04T14:20:38.699473427-07:00 level=info msg="Starting Grafana" version=9.4.3 commit=cf0a135595 branch=HEAD compiled=2023-03-02T12:28:42-08:00
logger=settings t=2023-05-04T14:20:38.70065999-07:00 level=warn msg="\"sentry\" frontend logging provider is deprecated and will be removed in the next major version. Use \"grafana\" provider instead."
logger=settings t=2023-05-04T14:20:38.700711657-07:00 level=info msg="Config loaded from" file=/usr/share/grafana/conf/defaults.ini
logger=settings t=2023-05-04T14:20:38.700719752-07:00 level=info msg="Config loaded from" file=/tmp/ray/session_latest/metrics/grafana/grafana.ini
logger=settings t=2023-05-04T14:20:38.7007243-07:00 level=info msg="Config overridden from command line" arg="default.paths.data=/grafana"
logger=settings t=2023-05-04T14:20:38.700728548-07:00 level=info msg="Config overridden from command line" arg="default.paths.logs=/var/log/grafana"
logger=settings t=2023-05-04T14:20:38.700733858-07:00 level=info

Now the job has been submitted, check on the job status

In [4]:
!sqs

JOBID            ST USER      NAME          NODES TIME_LIMIT       TIME  SUBMIT_TIME          QOS             START_TIME           FEATURES       NODELIST(REASON
259611           R  asnaylor  tmpwqmr3v1f   2          30:00       0:18  2023-05-04T14:20:37  gpu_debug       2023-05-04T14:20:37  gpu&a100&hbm40 nid[001080-0010


## Connect to Ray + Grafana dashboards

Go to the website:

In [5]:
rayCluster.ray_dashboard_url

'https://jupyter.nersc.gov/user/asnaylor/muller-shared-node-cpu/proxy/localhost:8265/'

## Connect to Ray Cluster

In [6]:
import ray

if ray.is_initialized:
    ray.shutdown()

ray.init(address='auto')

2023-05-04 14:20:57,460	INFO worker.py:1364 -- Connecting to existing Ray cluster at address: 128.55.173.46:6379...
2023-05-04 14:20:57,468	INFO worker.py:1544 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.15
Ray version:,2.3.0
Dashboard:,http://127.0.0.1:8265


Check all nodes connected to cluster

In [7]:
from nersc_cluster_deploy.ray import cluster_summary

cluster_summary()

Nodes: 3
CPU:   256
GPU:   8
RAM:   651.19 GB


## Setup PyTorch Model

In [9]:
import argparse
import os

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from filelock import FileLock
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18

import ray
import ray.train as train
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from ray.air.config import FailureConfig, RunConfig, ScalingConfig
from ray.train.torch import TorchTrainer
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.tune_config import TuneConfig
from ray.tune.tuner import Tuner

In [10]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) // session.get_world_size()
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def validate_epoch(dataloader, model, loss_fn):
    size = len(dataloader.dataset) // session.get_world_size()
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n "
        f"Accuracy: {(100 * correct):>0.1f}%, "
        f"Avg loss: {test_loss:>8f} \n"
    )
    return {"loss": test_loss}


def update_optimizer_config(optimizer, config):
    for param_group in optimizer.param_groups:
        for param, val in config.items():
            param_group[param] = val


def train_func(config):
    epochs = config.get("epochs", 3)

    model = resnet18()

    # Note that `prepare_model` needs to be called before setting optimizer.
    if not session.get_checkpoint():  # fresh start
        model = train.torch.prepare_model(model)

    # Create optimizer.
    optimizer_config = {
        "lr": config.get("lr"),
        "momentum": config.get("momentum"),
    }
    optimizer = torch.optim.SGD(model.parameters(), **optimizer_config)

    starting_epoch = 0
    if session.get_checkpoint():
        checkpoint_dict = session.get_checkpoint().to_dict()

        # Load in model
        model_state = checkpoint_dict["model"]
        model.load_state_dict(model_state)
        model = train.torch.prepare_model(model)

        # Load in optimizer
        optimizer_state = checkpoint_dict["optimizer_state_dict"]
        optimizer.load_state_dict(optimizer_state)

        # Optimizer configs (`lr`, `momentum`) are being mutated by PBT and passed in
        # through config, so we need to update the optimizer loaded from the checkpoint
        update_optimizer_config(optimizer, optimizer_config)

        # The current epoch increments the loaded epoch by 1
        checkpoint_epoch = checkpoint_dict["epoch"]
        starting_epoch = checkpoint_epoch + 1

    # Load in training and validation data.
    transform_train = transforms.Compose(
        [
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ]
    )  # meanstd transformation

    transform_test = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ]
    )

    data_dir = config.get("data_dir", os.path.expanduser("~/data"))
    os.makedirs(data_dir, exist_ok=True)
    with FileLock(os.path.join(data_dir, ".ray.lock")):
        train_dataset = CIFAR10(
            root=data_dir, train=True, download=True, transform=transform_train
        )
        validation_dataset = CIFAR10(
            root=data_dir, train=False, download=False, transform=transform_test
        )

    if config.get("test_mode"):
        train_dataset = Subset(train_dataset, list(range(64)))
        validation_dataset = Subset(validation_dataset, list(range(64)))

    worker_batch_size = config["batch_size"] // session.get_world_size()

    train_loader = DataLoader(train_dataset, batch_size=worker_batch_size)
    validation_loader = DataLoader(validation_dataset, batch_size=worker_batch_size)

    train_loader = train.torch.prepare_data_loader(train_loader)
    validation_loader = train.torch.prepare_data_loader(validation_loader)

    # Create loss.
    criterion = nn.CrossEntropyLoss()

    for epoch in range(starting_epoch, epochs):
        train_epoch(train_loader, model, criterion, optimizer)
        result = validate_epoch(validation_loader, model, criterion)
        checkpoint = Checkpoint.from_dict(
            {
                "epoch": epoch,
                "model": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
            }
        )

        session.report(result, checkpoint=checkpoint)


## Train Model

In [11]:
SCRATCH = os.getenv('SCRATCH')

node_resources = ray.cluster_resources()
num_workers = int(node_resources['GPU'])
use_gpu = True

data_dir = os.path.join(SCRATCH, 'CIFAR10')
num_epochs = 5
smoke_test = False
synch = False

In [12]:
trainer = TorchTrainer(
        train_func,
        scaling_config=ScalingConfig(
            num_workers=num_workers, use_gpu=use_gpu
        ),
    )

In [13]:
pbt_scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval=1,
        hyperparam_mutations={
            "train_loop_config": {
                # distribution for resampling
                "lr": tune.loguniform(0.001, 0.1),
                # allow perturbations within this set of categorical values
                "momentum": [0.8, 0.9, 0.99],
            }
        },
        synch=synch,
    )

In [14]:
tuner = Tuner(
        trainer,
        param_space={
            "train_loop_config": {
                "lr": tune.grid_search([0.001, 0.01, 0.05, 0.1]),
                "momentum": 0.8,
                "batch_size": 128 * num_workers,
                "test_mode": smoke_test,  # whether to to subset the data
                "data_dir": data_dir,
                "epochs": num_epochs,
            }
        },
        tune_config=TuneConfig(
            num_samples=1, metric="loss", mode="min", scheduler=pbt_scheduler
        ),
        run_config=RunConfig(
            stop={"training_iteration": 3 if smoke_test else num_epochs},
            failure_config=FailureConfig(max_failures=3),  # used for fault tolerance
        ),
    )


In [15]:
results = tuner.fit()

/global/homes/a/asnaylor/.local/muller/pytorch1.13.1/lib/python3.9/site-packages/ray/tune/tune.py:562: UserWarning: Consider boosting PBT performance by enabling `reuse_actors` as well as implementing `reset_config` for Trainable.
  warnings.warn(


2023-05-04 13:32:21,603	WARNING trial_runner.py:369 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (281 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent trials.
2023-05-04 13:32:21,608	WARNING trial_runner.py:1677 -- You are trying to access _search_alg interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for your scheduler to access this TrialRunner API, please reach out to Ray team on GitHub. A more strict API access pattern would be enforced starting 1.12s.0
(RayTrainWorker pid=24120, ip=128.55.173.55) 2023-05-04 13:32:32,343	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=8]
(RayTrainWorker pid=24120, ip=128.55.173.55) 2023-05-0

(RayTrainWorker pid=24120, ip=128.55.173.55) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /mscratch/sd/a/asnaylor/CIFAR10/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:03<00:00, 44986363.96it/s]


(RayTrainWorker pid=24120, ip=128.55.173.55) Extracting /mscratch/sd/a/asnaylor/CIFAR10/cifar-10-python.tar.gz to /mscratch/sd/a/asnaylor/CIFAR10
(RayTrainWorker pid=24121, ip=128.55.173.55) Files already downloaded and verified
(RayTrainWorker pid=110052, ip=128.55.173.56) Files already downloaded and verified
(RayTrainWorker pid=24122, ip=128.55.173.55) Files already downloaded and verified
(RayTrainWorker pid=24123, ip=128.55.173.55) Files already downloaded and verified
(RayTrainWorker pid=110051, ip=128.55.173.56) Files already downloaded and verified
(RayTrainWorker pid=110050, ip=128.55.173.56) Files already downloaded and verified
(RayTrainWorker pid=110049, ip=128.55.173.56) Files already downloaded and verified
(RayTrainWorker pid=110050, ip=128.55.173.56) loss: 7.209373  [    0/ 6250]
(RayTrainWorker pid=110049, ip=128.55.173.56) loss: 7.165215  [    0/ 6250]
(RayTrainWorker pid=110052, ip=128.55.173.56) loss: 7.214536  [    0/ 6250]
(RayTrainWorker pid=110051, ip=128.55.173

...

2023-05-04 13:40:38,368	WARNING util.py:244 -- The `process_trial_save` operation took 1.896 s, which may be a performance bottleneck.
(TorchTrainer pid=42580, ip=128.55.173.55) 2023-05-04 13:40:42,222	INFO trainable.py:791 -- Restored on 128.55.173.55 from checkpoint: /global/homes/a/asnaylor/ray_results/TorchTrainer_2023-05-04_13-32-20/TorchTrainer_c56f2_00002_2_lr=0.0500_2023-05-04_13-33-25/checkpoint_tmp293a46
(TorchTrainer pid=42580, ip=128.55.173.55) 2023-05-04 13:40:42,222	INFO trainable.py:800 -- Current state after restoring: {'_iteration': 4, '_timesteps_total': None, '_time_total': 79.69685482978821, '_episodes_total': None}
(RayTrainWorker pid=42758, ip=128.55.173.55) 2023-05-04 13:40:44,806	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=8]
(RayTrainWorker pid=125642, ip=128.55.173.56) 2023-05-04 13:40:47,955	INFO train_loop_utils.py:255 -- Moving model to device: cuda:3
(RayTrainWorker pid=42758, ip=128.55.173.55) 2023-05-04 13:40:47,981	INFO

(RayTrainWorker pid=42761, ip=128.55.173.55) Files already downloaded and verified
(RayTrainWorker pid=42759, ip=128.55.173.55) Files already downloaded and verified
(RayTrainWorker pid=42758, ip=128.55.173.55) Files already downloaded and verified
(RayTrainWorker pid=42760, ip=128.55.173.55) Files already downloaded and verified
(RayTrainWorker pid=125642, ip=128.55.173.56) Files already downloaded and verified
(RayTrainWorker pid=125640, ip=128.55.173.56) Files already downloaded and verified
(RayTrainWorker pid=125641, ip=128.55.173.56) Files already downloaded and verified
(RayTrainWorker pid=125639, ip=128.55.173.56) Files already downloaded and verified
(RayTrainWorker pid=125642, ip=128.55.173.56) loss: 1.213518  [    0/ 6250]
(RayTrainWorker pid=125641, ip=128.55.173.56) loss: 1.225929  [    0/ 6250]
(RayTrainWorker pid=125640, ip=128.55.173.56) loss: 1.199808  [    0/ 6250]
(RayTrainWorker pid=125639, ip=128.55.173.56) loss: 1.082679  [    0/ 6250]
(RayTrainWorker pid=42758, i

2023-05-04 13:41:04,545	WARNING util.py:244 -- The `process_trial_save` operation took 1.931 s, which may be a performance bottleneck.
(TorchTrainer pid=43635, ip=128.55.173.55) 2023-05-04 13:41:09,076	INFO trainable.py:791 -- Restored on 128.55.173.55 from checkpoint: /global/homes/a/asnaylor/ray_results/TorchTrainer_2023-05-04_13-32-20/TorchTrainer_c56f2_00003_3_lr=0.1000_2023-05-04_13-33-51/checkpoint_tmpd25ad5
(TorchTrainer pid=43635, ip=128.55.173.55) 2023-05-04 13:41:09,076	INFO trainable.py:800 -- Current state after restoring: {'_iteration': 4, '_timesteps_total': None, '_time_total': 79.40717196464539, '_episodes_total': None}
(RayTrainWorker pid=43740, ip=128.55.173.55) 2023-05-04 13:41:11,821	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=8]
(RayTrainWorker pid=43740, ip=128.55.173.55) 2023-05-04 13:41:14,896	INFO train_loop_utils.py:255 -- Moving model to device: cuda:2
(RayTrainWorker pid=127069, ip=128.55.173.56) 2023-05-04 13:41:15,003	INFO

(RayTrainWorker pid=43742, ip=128.55.173.55) Files already downloaded and verified
(RayTrainWorker pid=43740, ip=128.55.173.55) Files already downloaded and verified
(RayTrainWorker pid=127070, ip=128.55.173.56) Files already downloaded and verified
(RayTrainWorker pid=127069, ip=128.55.173.56) Files already downloaded and verified
(RayTrainWorker pid=127071, ip=128.55.173.56) Files already downloaded and verified
(RayTrainWorker pid=127072, ip=128.55.173.56) Files already downloaded and verified
(RayTrainWorker pid=43741, ip=128.55.173.55) Files already downloaded and verified
(RayTrainWorker pid=43743, ip=128.55.173.55) Files already downloaded and verified
(RayTrainWorker pid=43742, ip=128.55.173.55) loss: 1.087673  [    0/ 6250]
(RayTrainWorker pid=43743, ip=128.55.173.55) loss: 1.226189  [    0/ 6250]
(RayTrainWorker pid=43741, ip=128.55.173.55) loss: 1.255771  [    0/ 6250]
(RayTrainWorker pid=43740, ip=128.55.173.55) loss: 1.033957  [    0/ 6250]
(RayTrainWorker pid=127072, ip=1

2023-05-04 13:41:31,204	WARNING util.py:244 -- The `process_trial_save` operation took 1.901 s, which may be a performance bottleneck.
2023-05-04 13:41:32,182	INFO tune.py:798 -- Total run time: 550.63 seconds (550.57 seconds for the tuning loop).


In [16]:
print(results.get_best_result(metric="loss", mode="min"))

Result(metrics={'loss': 0.9780136168003082, '_timestamp': 1683232836, '_time_this_iter_s': 15.813694477081299, '_training_iteration': 1, 'should_checkpoint': True, 'done': True, 'trial_id': 'c56f2_00001', 'experiment_tag': '1_lr=0.0100@perturbed[train_loop_config=lr_0_04800000000000001_momentum_0_8_batch_size_1024_test_mode_False_data_dir_mscratch_sd_a_asnaylor_CIFAR10_epochs_5]'}, error=None, log_dir=PosixPath('/global/homes/a/asnaylor/ray_results/TorchTrainer_2023-05-04_13-32-20/TorchTrainer_c56f2_00001_1_lr=0.0100_2023-05-04_13-33-00'))


## Close cluster conection and stop job

In [8]:
ray.shutdown()

In [10]:
rayCluster.shutdown()

2023-05-04 14:21:36,233 INFO <Service-Manager> Shutdown cluster
2023-05-04 14:21:36,234 INFO <Grafana> Stopping service
2023-05-04 14:21:36,235 INFO <Prometheus> Stopping service
2023-05-04 14:21:36,236 INFO <RayHead> Stopping service


In [9]:
!scancel -u "$USER"

## Explore Training in Tensorboard

In [39]:
import nersc_tensorboard_helper
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [40]:
log_dir = str(results.get_best_result(metric="loss", mode="min").log_dir)

In [41]:
%tensorboard --logdir $log_dir --port 0

In [42]:
nersc_tensorboard_helper.tb_address()